In [1]:
import pandas as pd
import openai
import pandas as pd
import numpy as np
import glob
from tqdm import tqdm
import json
import os
from dotenv import load_dotenv
import time

In [2]:
load_dotenv()
openai.api_key = os.environ['OPENAI_API']

In [3]:
def create_clean_paragraphs(input_dict):

    # Create a new dictionary to store the cleaned-up values
    output_dict = {}
    
    # Loop through the keys and values of the input dictionary
    for key, value in input_dict.items():
        # Convert the list of values to a set to remove duplicates
        unique_values = set(value)
        
        # Join the sentences together into a single string
        combined_string = ' '.join(unique_values)
        
        # Add the cleaned-up string to the output dictionary
        output_dict[key] = combined_string
    
    # Return the cleaned-up dictionary
    return output_dict

In [4]:
folder_survey = "../../../data/surveys/"
file = "df_survey_result.csv"
df = pd.read_csv(F"{folder_survey}{file}")
# df = df.set_index("Species")
df = df.iloc[: , 1:]
# df = df.reset_index()

### Caribbean

In [5]:
df_car = df[df['Dataset'] == 'Caribbean']
df_car.head(10)

,Species,Main Trait,SIM,Dataset,Sentence,Result
396,Bursera simaruba,Leaf glands,Jacc,Caribbean,"Leaf venation: brachidodrome, pinnate.",Can infer correct Entity
397,Bursera simaruba,Leaf rachis,Jacc,Caribbean,"Leaf venation: brachidodrome, pinnate.",Can infer correct Entity
398,Crossopetalum rhacoma,Fruit type,Jacc,Caribbean,"Fruit a drupe, seed exarillate.",Can infer correct Quality
399,Randia aculeata,Leaf margin,Bert,Caribbean,Leaf margin: entire.,Can infer correct Value
400,Vachellia tortuosa,Leaf upper side,Jacc,Caribbean,Glands are usually present on the rachis and t...,None of the above
401,Pithecellobium unguis-cati,Leaf position,Jacc,Caribbean,Alternate hairless leaves have petioles 2 to 5...,Can infer correct Value
402,Libidibia coriaria,Leaf rachis,Jacc,Caribbean,"Perennial, Not climbing, Shrub/Tree.",None of the above
403,Guaiacum officinale,Fruit colour,Bert,Caribbean,Mature: Light Green to Medium Bright Green.,None of the above
404,Avicennia germinans,Seed colour,Jacc,Caribbean,"Seed 1, large, flattened, often germinating on...",Can infer correct Entity
405,Schoepfia schreberi,Fruit shape,Bert,Caribbean,"Spurious fruit ellipsoid, 10-13 mm.",Can infer correct Quality


In [6]:
# Define the path to the directory where the prompts and results will be saved
folder_prompts = "../../../data/surveys/PromptResultsSentences/"

# Loop over species


for idx in tqdm(range(0, df_car.index.shape[0])):

    if idx > 1:
        continue

    species = df_car.iloc[idx, 0]
    trait = df_car.iloc[idx, 1]
    dataset = df_car.iloc[idx, 3]
    sentence = df_car.iloc[idx, 4]
    
    # Replace spaces in the species name with underscores
    folder_species = species.replace(' ', '_')

    # Create the question and options for the ChatGPT prompt
    user_content = F"""Can you help me determining of the sentence tells you anything about the '{trait}'?"""

    # Combine the words in the trait name with capital letters and use this as the file name
    file_name = F"{idx}_{dataset}_{folder_species}"
    
    # Check if file is already there (OpenAI Outage)
    if os.path.exists(F"{folder_prompts}{file_name}.json"):
        continue

    # Create the messages to send to the ChatGPT API
    messages = [
        {"role": "user", "content": sentence},
        {"role": "user", "content": user_content}
        ]
    # Call the ChatGPT API to generate a completion for the prompt
    completion = openai.ChatCompletion.create(
        model = "gpt-3.5-turbo",
        messages = messages,
    )

    # Lower pressure on API?
    time.sleep(2)

    # Save the completion to a JSON file with the file name in the directory
    with open(F"{folder_prompts}{file_name}.json", 'w') as fp:
        json.dump(completion, fp)

100%|██████████| 1/1 [00:04<00:00,  4.50s/it]


In [12]:
df_car.index.shape[0]

434

In [ ]:
idx = 81

print(df_car.iloc[idx, 1])
print(df_car.iloc[idx, 4])
print(df_car.iloc[idx, 5])

In [ ]:
sentence = df_car.iloc[idx, 4]
trait = df_car.iloc[idx, 1]

user_content = F"""Can you help me determining of the sentence tells you anything about the '{trait}'?"""

In [ ]:
print(user_content)

In [ ]:
messages = [
    {"role": "user", "content": sentence},
    {"role": "user", "content": user_content}
    ]
# Call the ChatGPT API to generate a completion for the prompt
completion = openai.ChatCompletion.create(
    model = "gpt-3.5-turbo",
    messages = messages,
)

In [ ]:
completion

In [ ]:
sentence = df_car.loc[111, 'Sentence']
trait = df_car.loc[111, 'Main Trait']

user_content = F"""
Can you help us with determining if, using only the provided text, it would be possible to infer the entity, quality or value from the sentence for the trait: '{trait}'?

 There are four options:
	- Can infer correct Entity
	- Can infer correct Quality
	- Can infer correct Value
	- None of the above

For each choice we have a small example:
	- Correct Entity: e.g. crown, flower or fruit.
	- Correct Quality: e.g. forest layer, number of petals or colour.
	- Correct Value: e.g. canopy, 5 or red.
"""

In [ ]:
print(sentence)
print(user_content)

In [ ]:
messages = [
    {"role": "user", "content": sentence},
    {"role": "user", "content": user_content}
    ]
# Call the ChatGPT API to generate a completion for the prompt
completion = openai.ChatCompletion.create(
    model = "gpt-3.5-turbo",
    messages = messages,
)

In [ ]:
completion

In [ ]:
# Define the path to the directory where the prompts and results will be saved
folder_prompts = "../../../data/OpenAI/PromptsResults/ZeroShot/"

species_lst = df_Andrei.index
trait_dict = create_trait_dict(df_Andrei)
traits = list(trait_dict.keys())

# Loop over species
for idx, species in enumerate(species_lst[0:]):
   
    # Replace spaces in the species name with underscores
    folder_species = species.replace(' ', '_')

    # Try to create a directory for the prompts for the species
    try:
        os.makedirs(F"{folder_prompts}{folder_species}")
    except FileExistsError:
        pass

    for trait in (pbar := tqdm(traits[0:], leave=False, position=0)):
        pbar.set_description(f"{idx}: {species}")

        # Get the options for the trait
        options = trait_dict[trait]

        # Create the question and options for the ChatGPT prompt
        question = F"For the species '{species}' which of the following options is applicable for the trait '{trait}'" 
        text_helper = F"Return 'NaN' if none of the options are applicable. Please return your answer as a Python list."
        user_content = F"{question}\n{options}\n{text_helper}"

        # Combine the words in the trait name with capital letters and use this as the file name
        file_name = combine_words_with_capital(trait)
        
        # Check if file is already there (OpenAI Outage)
        if os.path.exists(F"{folder_prompts}{folder_species}/{file_name}.json"):
            continue

        # # Create the messages to send to the ChatGPT API
        messages = [
            {"role": "user", "content": sentence},
            {"role": "user", "content": user_content}
            ]
        # Call the ChatGPT API to generate a completion for the prompt
        completion = openai.ChatCompletion.create(
            model = "gpt-3.5-turbo",
            messages = messages,
        )

        # Lower pressure on API?
        time.sleep(2)

        # Save the completion to a JSON file with the file name in the directory
        with open(F"{folder_prompts}{folder_species}/{file_name}.json", 'w') as fp:
            json.dump(completion, fp)
            